####### NETFLIX Extraction

In [2]:
import imaplib
import email
import re
import pandas as pd
from bs4 import BeautifulSoup

# User credentials (use App Password for Gmail)
EMAIL = "tanuja.ch076@gmail.com"
PASSWORD = "omip uivi tqxc owpa"

# Connect to Gmail IMAP server
try:
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login(EMAIL, PASSWORD)
    print("✅ Successfully connected to Gmail.")
except Exception as e:
    print(f"❗ Connection failed: {e}")
    exit()

# Select INBOX folder
try:
    mail.select("INBOX")
    print("✅ INBOX folder selected.")
except Exception as e:
    print(f"❗ Failed to select INBOX: {e}")
    exit()

# Search for Netflix emails
try:
    search_query = '(OR (SUBJECT "Netflix Subscription Renewal") (SUBJECT "Netflix Renewal Notice") (BODY "Netflix"))'
    print(f"🔎 Searching for Netflix emails...")
    status, messages = mail.search(None, search_query)

    if status != "OK" or not messages[0].split():
        print("❗ No Netflix emails found.")
        exit()

    print(f"✅ Found {len(messages[0].split())} Netflix emails.")
except Exception as e:
    print(f"❗ Search failed: {e}")
    exit()

# Extract Netflix subscription details
subscription_data = []

for num in messages[0].split():
    try:
        status, msg_data = mail.fetch(num, "(RFC822)")
        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)
    except Exception as e:
        print(f"❗ Failed to fetch email: {e}")
        continue

    print(f"📧 Processing email with subject: {msg['Subject']}")

    # Extract email content
    email_body = ""
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            if content_type in ["text/plain", "text/html"]:
                try:
                    body_content = part.get_payload(decode=True).decode('utf-8', errors='ignore')
                    if content_type == "text/html":
                        soup = BeautifulSoup(body_content, 'html.parser')
                        email_body = soup.get_text()
                    else:
                        email_body = body_content
                    break
                except Exception as e:
                    print(f"❗ Failed to decode content: {e}")
    else:
        try:
            email_body = msg.get_payload(decode=True).decode('utf-8', errors='ignore')
        except Exception as e:
            print(f"❗ Failed to decode plain email content: {e}")

    # Extract necessary details using regex
    invoice_no = re.search(r"Invoice No\.\s*([A-Z0-9-]+)", email_body)
    date = re.search(r"Date\s*(\d{2}/\d{2}/\d{4})", email_body)
    Total = re.search(r"Total\s*₹\s*([\d,]+\.?\d*)", email_body)

    # Append extracted data
    subscription_data.append({
        "User Email": EMAIL,
        "Streaming Service": "Netflix",
        "Invoice No.": invoice_no.group(1) if invoice_no else "Not Found",
        "Date": date.group(1) if date else "Not Found",
        "Total (INR)": float(Total.group(1).replace(',', '')) if Total else 0.0,
    })
    print("✅ Netflix details extracted.")

# Save data to CSV
if subscription_data:
    df = pd.DataFrame(subscription_data)
    df.to_csv("netflix_subscription_details.csv", index=False)
    print("✅ Data saved to 'netflix_subscription_details.csv'.")
    print(df.head())
else:
    print("❗ No Netflix subscription data extracted.")

# Logout and close connection
mail.logout()
print("✅ Disconnected from Gmail.")


✅ Successfully connected to Gmail.
✅ INBOX folder selected.
🔎 Searching for Netflix emails...
✅ Found 4 Netflix emails.
📧 Processing email with subject: Netflix Subscription Renewal
✅ Netflix details extracted.
📧 Processing email with subject: Netflix Subscription Renewal
✅ Netflix details extracted.
📧 Processing email with subject: Netflix Subscription Renewal
✅ Netflix details extracted.
📧 Processing email with subject: Netflix Renewal Notice
✅ Netflix details extracted.
✅ Data saved to 'netflix_subscription_details.csv'.
               User Email Streaming Service              Invoice No.  \
0  tanuja.ch076@gmail.com           Netflix  C21E2-1A863-8FF45-180A6   
1  tanuja.ch076@gmail.com           Netflix  C21E2-1A863-8FF45-180A6   
2  tanuja.ch076@gmail.com           Netflix  C21E2-1A863-8FF45-180A6   
3  tanuja.ch076@gmail.com           Netflix  C21E2-1A863-8FF45-180A6   

        Date  Total (INR)  
0  Not Found        199.0  
1  Not Found        199.0  
2  Not Found        199.0

####### Spotify and Swiggy

In [3]:
import imaplib
import email
import re
import pandas as pd
from bs4 import BeautifulSoup

# User credentials (use App Password for Gmail)
EMAIL = "tanuja.ch076@gmail.com"
PASSWORD = "omip uivi tqxc owpa"

# Connect to Gmail IMAP server
try:
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login(EMAIL, PASSWORD)
    print("✅ Successfully connected to Gmail.")
except Exception as e:
    print(f"❗ Connection failed: {e}")
    exit()

# Select All Mail to ensure all folders are checked
try:
    mail.select('"[Gmail]/All Mail"')
    print("✅ All Mail folder selected.")
except Exception as e:
    print(f"❗ Failed to select mail folder: {e}")
    exit()

# Search for emails related to Swiggy, Spotify, Netflix, or Apple Music
try:
    search_query = '(OR (SUBJECT "Swiggy One Lite") (SUBJECT "Spotify Premium") (SUBJECT "Netflix Subscription Renewal") (SUBJECT "Apple Music Subscription Renewal"))'
    print(f"🔎 Searching emails using query: {search_query}")
    status, messages = mail.search(None, search_query)
    
    print(f"IMAP Status: {status}")
    print(f"Messages: {messages}")
    
    if status != "OK" or not messages[0].split():
        print("❗ No relevant emails found. Trying other searches...")
        
        # Search by Body Text (if supported)
        search_query_body = '(OR (BODY "membership") (BODY "receipt") (BODY "subscription"))'
        status, messages = mail.search(None, search_query_body)
        
        if status != "OK" or not messages[0].split():
            print("❗ Still no emails found using BODY search.")
            
            # Search using a simpler query
            search_query_simple = 'ALL'
            status, messages = mail.search(None, search_query_simple)
            print(f"✅ Total emails found using ALL query: {len(messages[0].split())}")

        else:
            print(f"✅ Found {len(messages[0].split())} emails using BODY search.")
    else:
        print(f"✅ {len(messages[0].split())} relevant emails found.")
except Exception as e:
    print(f"❗ Search failed: {e}")
    exit()

# Process emails
subscription_data = []

for num in messages[0].split():
    try:
        status, msg_data = mail.fetch(num, "(RFC822)")
        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)
    except Exception as e:
        print(f"❗ Failed to fetch email: {e}")
        continue

    print(f"📧 Processing email with subject: {msg['Subject']}")
    
    # Extract email content
    email_body = ""
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            try:
                if content_type == "text/plain":
                    email_body = part.get_payload(decode=True).decode('utf-8', errors='ignore')
                    break
                elif content_type == "text/html":
                    html_content = part.get_payload(decode=True).decode('utf-8', errors='ignore')
                    soup = BeautifulSoup(html_content, 'html.parser')
                    email_body = soup.get_text()
                    break
            except Exception as e:
                print(f"❗ Failed to decode email content: {e}")
    else:
        try:
            email_body = msg.get_payload(decode=True).decode('utf-8', errors='ignore')
        except Exception as e:
            print(f"❗ Failed to decode plain email content: {e}")

    # Extract details for Swiggy One Lite
    if "Swiggy One Lite" in email_body:
        print("📧 Processing Swiggy One Lite Email...")

        date_of_email = re.search(r"Date of Email\s*(\d{2}-\d{2}-\d{4})", email_body)
        plan_tenure = re.search(r"Plan Tenure\s*([\w\s]+)", email_body)
        plan_valid_from = re.search(r"Plan Valid From\s*(\d{2}-\d{2}-\d{4})", email_body)
        plan_valid_till = re.search(r"Plan Valid Till\s*(\d{2}-\d{2}-\d{4})", email_body)
        amount_paid = re.search(r"Amount Paid.*₹\s*([\d,]+\.?\d*)", email_body)
        transaction_id = re.search(r"Transaction Id\s*([\d]+)", email_body)

        if date_of_email and plan_tenure and plan_valid_from and plan_valid_till and amount_paid and transaction_id:
            subscription_data.append({
                "User Email": EMAIL,
                "Platform": "Swiggy One Lite",
                "Date of Email": date_of_email.group(1),
                "Plan Tenure": plan_tenure.group(1),
                "Plan Valid From": plan_valid_from.group(1),
                "Plan Valid Till": plan_valid_till.group(1),
                "Amount Paid (INR)": float(amount_paid.group(1).replace(',', '')),
                "Transaction ID": transaction_id.group(1)
            })
            print("✅ Swiggy One Lite details extracted.")

    # Extract details for Spotify Premium
    elif "Spotify Premium" in email_body:
        print("📧 Processing Spotify Premium Email...")

        order_id = re.search(r"Order ID:\s*([\d-]+)", email_body)
        amount = re.search(r"Spotify Premium\s*([\d,]+\.?\d*)\s*INR", email_body)
        duration = re.search(r"([\d]+ months)", email_body)
        igst = re.search(r"IGST \(18%\)\s*([\d,]+\.?\d*)\s*INR", email_body)
        total = re.search(r"Total\s*([\d,]+\.?\d*)\s*INR", email_body)

        if order_id and amount and duration and igst and total:
            subscription_data.append({
                "User Email": EMAIL,
                "Platform": "Spotify Premium",
                "Order ID": order_id.group(1),
                "Subscription Cost (INR)": float(amount.group(1).replace(',', '')),
                "Duration": duration.group(1),
                "IGST (INR)": float(igst.group(1).replace(',', '')),
                "Total (INR)": float(total.group(1).replace(',', ''))
            })
            print("✅ Spotify Premium details extracted.")

    # Extract details for Apple Music
    elif "Apple Music" in email_body:
        print("📧 Processing Apple Music Email...")

        plan_type = re.search(r"Apple Music\s*\n*(\w+\s*\(\d+\s*month[s]*\))", email_body)
        amount = re.search(r"₹\s*([\d,]+\.?\d*)/month", email_body)
        renewal_date = re.search(r"Starting from (\d{2} \w+ \d{4})", email_body)

        if plan_type and amount and renewal_date:
            subscription_data.append({
                "User Email": EMAIL,
                "Platform": "Apple Music",
                "Plan Type": plan_type.group(1),
                "Amount Per Month (INR)": float(amount.group(1).replace(',', '')),
                "Renewal Date": renewal_date.group(1)
            })
            print("✅ Apple Music details extracted.")

# Save data to CSV
if subscription_data:
    df = pd.DataFrame(subscription_data)
    df.to_csv("subscription_details.csv", index=False)
    print("✅ Data saved to 'subscription_details.csv'.")
    print(df.head())
else:
    print("❗ No subscription data extracted.")

# Logout and close connection
mail.logout()
print("✅ Disconnected from Gmail.")


✅ Successfully connected to Gmail.
✅ All Mail folder selected.
🔎 Searching emails using query: (OR (SUBJECT "Swiggy One Lite") (SUBJECT "Spotify Premium") (SUBJECT "Netflix Subscription Renewal") (SUBJECT "Apple Music Subscription Renewal"))
IMAP Status: OK
Messages: [b'']
❗ No relevant emails found. Trying other searches...
✅ Found 9 emails using BODY search.
📧 Processing email with subject: Spotify Receipt
📧 Processing Spotify Premium Email...
📧 Processing email with subject: Your Google Play Order Receipt from Dec 16, 2024
📧 Processing email with subject: Spotify Receipt
📧 Processing Spotify Premium Email...
✅ Spotify Premium details extracted.
📧 Processing email with subject: Spotify Premium receipt
📧 Processing Spotify Premium Email...
✅ Spotify Premium details extracted.
📧 Processing email with subject: Spotify Premium Receipt
📧 Processing Spotify Premium Email...
✅ Spotify Premium details extracted.
📧 Processing email with subject: Spotify Premium Receipt
📧 Processing Spotify Pr

####### All three

In [4]:
import imaplib
import email
import re
import pandas as pd
from bs4 import BeautifulSoup

# User credentials (use App Password for Gmail)
EMAIL = "tanuja.ch076@gmail.com"
PASSWORD = "omip uivi tqxc owpa"

# Connect to Gmail IMAP server
try:
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login(EMAIL, PASSWORD)
    print("✅ Successfully connected to Gmail.")
except Exception as e:
    print(f"❗ Connection failed: {e}")
    exit()

# Select All Mail to ensure all folders are checked
try:
    mail.select('"[Gmail]/All Mail"')
    print("✅ All Mail folder selected.")
except Exception as e:
    print(f"❗ Failed to select mail folder: {e}")
    exit()

# Search for emails related to Swiggy, Spotify, Netflix, or Apple Music
try:
    search_query = '(OR (SUBJECT "Swiggy One Lite") (SUBJECT "Spotify Premium") (SUBJECT "Netflix Subscription Renewal") (SUBJECT "Netflix Renewal Notice") )'
    print(f"🔎 Searching emails using query: {search_query}")
    status, messages = mail.search(None, search_query)
    
    print(f"IMAP Status: {status}")
    print(f"Messages: {messages}")
    
    if status != "OK" or not messages[0].split():
        print("❗ No relevant emails found. Trying other searches...")
        
        # Search by Body Text (if supported)
        search_query_body = '(OR (BODY "membership") (BODY "receipt") (BODY "subscription"))'
        status, messages = mail.search(None, search_query_body)
        
        if status != "OK" or not messages[0].split():
            print("❗ Still no emails found using BODY search.")
            
            # Search using a simpler query
            search_query_simple = 'ALL'
            status, messages = mail.search(None, search_query_simple)
            print(f"✅ Total emails found using ALL query: {len(messages[0].split())}")

        else:
            print(f"✅ Found {len(messages[0].split())} emails using BODY search.")
    else:
        print(f"✅ {len(messages[0].split())} relevant emails found.")
except Exception as e:
    print(f"❗ Search failed: {e}")
    exit()

# Process emails
subscription_data = []

for num in messages[0].split():
    try:
        status, msg_data = mail.fetch(num, "(RFC822)")
        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)
    except Exception as e:
        print(f"❗ Failed to fetch email: {e}")
        continue

    print(f"📧 Processing email with subject: {msg['Subject']}")
    
    # Extract email content
    email_body = ""
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            try:
                if content_type == "text/plain":
                    email_body = part.get_payload(decode=True).decode('utf-8', errors='ignore')
                    break
                elif content_type == "text/html":
                    html_content = part.get_payload(decode=True).decode('utf-8', errors='ignore')
                    soup = BeautifulSoup(html_content, 'html.parser')
                    email_body = soup.get_text()
                    break
            except Exception as e:
                print(f"❗ Failed to decode email content: {e}")
    else:
        try:
            email_body = msg.get_payload(decode=True).decode('utf-8', errors='ignore')
        except Exception as e:
            print(f"❗ Failed to decode plain email content: {e}")

    # Extract details for Swiggy One Lite
    if "Swiggy One Lite" in email_body:
        print("📧 Processing Swiggy One Lite Email...")

        date_of_email = re.search(r"Date of Email\s*(\d{2}-\d{2}-\d{4})", email_body)
        amount_paid = re.search(r"Amount Paid.*₹\s*([\d,]+\.?\d*)", email_body)
        transaction_id = re.search(r"Transaction Id\s*([\d]+)", email_body)

        if date_of_email and amount_paid and transaction_id:
            subscription_data.append({
                "User Email": EMAIL,
                "Platform": "Swiggy One Lite",
                "Transaction ID": transaction_id.group(1),
                "Date of Email": date_of_email.group(1),
                "Total (INR)": float(amount_paid.group(1).replace(',', ''))
                
            })
            print("✅ Swiggy One Lite details extracted.")

    # Extract details for Spotify Premium
    elif "Spotify Premium" in email_body:
        print("📧 Processing Spotify Premium Email...")

        order_id = re.search(r"Order ID:\s*([\d-]+)", email_body)
        duration = re.search(r"([\d]+ months)", email_body)
        total = re.search(r"Total\s*([\d,]+\.?\d*)\s*INR", email_body)

        if order_id and duration and total:
            subscription_data.append({
                "User Email": EMAIL,
                "Platform": "Spotify Premium",
                "Transaction ID": order_id.group(1),
                "Duration": duration.group(1),
                "Total (INR)": float(total.group(1).replace(',', ''))
            })
            print("✅ Spotify Premium details extracted.")
    elif "Netflix" in email_body:
        print("📧 Processing Netflix Email...")
        invoice_no = re.search(r"Invoice No\.\s*([A-Z0-9-]+)", email_body)
        total = re.search(r"Total\s*₹\s*([\d,]+\.?\d*)", email_body)

        if all([invoice_no, total]):
            subscription_data.append({
                "User Email": EMAIL,
                "Platform": "Netflix",
                "Transaction ID": invoice_no.group(1),                
                "Total (INR)": float(total.group(1).replace(',', ''))
            })
            print("✅ Netflix details extracted.")

    
   

# Save data to CSV
if subscription_data:
    df = pd.DataFrame(subscription_data)
    df.to_csv("subscription_details.csv", index=False)
    print("✅ Data saved to 'subscription_details.csv'.")
    print(df.head())
else:
    print("❗ No subscription data extracted.")

# Logout and close connection
mail.logout()
print("✅ Disconnected from Gmail.")


✅ Successfully connected to Gmail.
✅ All Mail folder selected.
🔎 Searching emails using query: (OR (SUBJECT "Swiggy One Lite") (SUBJECT "Spotify Premium") (SUBJECT "Netflix Subscription Renewal") (SUBJECT "Netflix Renewal Notice") (SUBJECT "Apple Music Subscription Renewal"))
IMAP Status: OK
Messages: [b'']
❗ No relevant emails found. Trying other searches...
✅ Found 9 emails using BODY search.
📧 Processing email with subject: Spotify Receipt
📧 Processing Spotify Premium Email...
📧 Processing email with subject: Your Google Play Order Receipt from Dec 16, 2024
📧 Processing email with subject: Spotify Receipt
📧 Processing Spotify Premium Email...
✅ Spotify Premium details extracted.
📧 Processing email with subject: Spotify Premium receipt
📧 Processing Spotify Premium Email...
✅ Spotify Premium details extracted.
📧 Processing email with subject: Spotify Premium Receipt
📧 Processing Spotify Premium Email...
✅ Spotify Premium details extracted.
📧 Processing email with subject: Spotify Prem